# COGS 108 Final Project

## Group Members: 
- A14178715
- A11475687
- A12594395
- A11729103

### Introduction and Background:

What would it look like if we visualized a predicted sea level rise for the future? How accurate can we get based on a large dataset of the sea level rising over a long period of time?

Well, we should see a fairly accurate model of future sea level, represented by a map of the coastal regions of California flooded. This model will get less and less accurate the further we go into the future. To start, we need data. Here is a good one:

ftp://podaac.jpl.nasa.gov/allData/merged_alt/L2/TP_J1_OSTM/global_mean_sea_level/GMSL_TPJAOS_V4_199209_201702.txt

ftp://podaac.jpl.nasa.gov/allData/merged_alt/L2/TP_J1_OSTM/global_mean_sea_level/GMSL_TPJAOS_V4_199209_201703.txt (updated link, we downloaded the data from the first link while it still worked)

This is NASA's data on calculating the sea level increase with satellites. The link worked a while back, but no longer works, so we downloaded the data while it was available and have included it in our repository.


### Data Description:

NASA's description of the data:

"Global Mean Sea Level Data"
This file contains Global Mean Sea Level (GMSL) variations computed at the NASA Goddard Space Flight Center under the 
auspices of the NASA MEaSUREs program. The GMSL was generated using the Integrated Multi-Mission Ocean Altimeter Data for Climate Research (http://podaac.jpl.nasa.gov/dataset/MERGED_TP_J1_OSTM_OST_ALL_V4). It combines Sea Surface Heights from TOPEX/Poseidon, Jason-1, OSTM/Jason-2, and Jason-3 to a common terrestrial reference frame with all inter-mission biases, range and geophysical corrections applied and placed onto a georeferenced orbit. This creates a consistent data record throughout time, regardless of the instrument used."

### Starting out: imports

In [1]:
# imports
import csv
import itertools
import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline
  

### Data Cleaning and Pre-processing

In [10]:
txtFile = "nasa_data_noheader.txt"
csvFile = "data.csv"
def processTxt(txtFile):
    with open(txtFile,'r') as in_file:
        # Our lines in nasa data have 1-4 spaces of separation so first we split the line
        stripped = (line.strip() for line in in_file)
        lines = (line.split(" ") for line in stripped if line)
        
        # Then here we write it to the csv and if there is an empty space it will replace it with nothing 
        with open(csvFile,'w') as out_file:
            # Column titles (adds empty row 2 we can just delete once its in pandas dataframe)
            writer = csv.writer(out_file)
            writer.writerow(('altimeter_type', 'file_cycle_#','date','obs_#','weighted_obs_#','mean_variation','std_dev_of_mean_var','smoothed_variation1','smoothed_variation2','std_dev_of_mean_var2','smoothed_variation3','smoothed_variation4', 'col12'))
            # for the above line, there are many extra and redundant forms of the data, such as GIA applied vs not applied. These columns are explained in the data.
            # four ur purposes, we only need a few of these columns, such as date, so the rest were not labelled in detail.
            for line in lines:
                for index in line:
                    # Replacing the white space with nothing
                    index.replace(" ", "") 
                    if index != "":
                        out_file.write(index + ",")
                out_file.write('\n')
processTxt(txtFile)


df = pd.read_csv('data.csv')
# remove the extra row (row2), drops only if all col = nan
df = df.dropna(how = 'all')

# hardcode way
# df = df.drop(df.index[1])
# only do this if you're absolutely sure the second row is all NaN. It looks fine to me in the original data file. 

# make a new dataframe without the columns we don't need
df2 = df[['date', 'mean_variation', 'smoothed_variation3']].copy()

# print the dataframe so we know we are on the right track
df2

# because the data point in every column of variation starts negative, we need to make them start at 0 for pretty plotting. 
# add 40.28 to every data point in the mean_variation column and 40.63 to every column in the smoothed_variation3 column
# this brings the floor of our data up to get it to look more like the plot on NASA's page: 
# https://climate.nasa.gov/vital-signs/sea-level/


# lastly, change the dates to human readable dates that are easier to plot, like NASA did for their plot. 
# the dates are in fractions of years. There are many ways of converting them in python, just search "convert fractional years to real dates in python"

date  mean_variation  smoothed_variation3
0    1993.011475          -40.28               -40.63
1    1993.038696          -44.20               -41.89
2    1993.065918          -44.17               -42.02
3    1993.093018          -46.32               -42.36
4    1993.120239          -40.91               -41.57
5    1993.147339          -39.32               -40.50
6    1993.174561          -38.86               -39.52
7    1993.201660          -37.62               -38.80
8    1993.228882          -37.80               -38.39
9    1993.255981          -45.73               -38.27
10   1993.283203          -39.55               -38.42
11   1993.310303          -36.22               -38.80
12   1993.337524          -38.38               -39.37
13   1993.364746          -41.33               -39.85
14   1993.391846          -42.35               -40.35
15   1993.419067          -42.08               -40.29
16   1993.446167          -39.13               -40.27
17   1993.473389          -39.16               -40.06
18   1993.500488          -39.82               -39.91
19   1993.527710          -41.09               -39.73
20   1993.554810          -37.32               -39.32
21   1993.582031          -39.97               -38.60
22   1993.609131          -39.50               -37.68
23   1993.636352          -35.68               -36.76
24   1993.663574          -34.51               -35.96
25   1993.690674          -36.13               -35.18
26   1993.717895          -37.06               -34.39
27   1993.744995          -33.18               -33.53
28   1993.772217          -29.57               -32.89
29   1993.799316          -31.83               -32.75
..           ...             ...                  ...
858  2016.332642           33.45                40.79
859  2016.359741           31.97                40.32
860  2016.386841           31.72                40.38
861  2016.413940           35.32                40.82
862  2016.441040           35.33                41.37
863  2016.468140           36.79                41.75
864  2016.495239           35.73                41.84
865  2016.522339           35.12                41.72
866  2016.549316           35.46                41.56
867  2016.576416           33.80                41.61
868  2016.603516           33.79                42.11
869  2016.630615           34.00                43.11
870  2016.657715           39.34                44.38
871  2016.684814           41.43                45.55
872  2016.711914           41.84                46.41
873  2016.739014           39.32                47.01
874  2016.766113           38.70                47.62
875  2016.793213           42.33                48.38
876  2016.820312           43.62                49.21
877  2016.847412           43.68                49.84
878  2016.874512           44.86                50.01
879  2016.901611           46.29                49.56
880  2016.928711           42.17                48.53
881  2016.955811           39.61                47.16
882  2016.982788           39.14                45.79
883  2017.010010           36.94                44.72
884  2017.037109           37.17                43.96
885  2017.064331           34.80                43.46
886  2017.091431           33.46                43.14
887  2017.118652           33.39                42.94

[888 rows x 3 columns]

### Data Visualization

In [11]:
# plot data using plt.something()


# this is not needed but looks interesting
# pd.scatter_matrix(df2)

In [ ]:
# predict future sea level rise for the next 100 years

In [ ]:
# show what this prediction looks like in a google map

### Conclusions and Discussion:

the end